## Problem Description
In the year 2026, Metropolis has fully integrated autonomous drone delivery. As a lead software engineer, your task is to develop the navigation system for a delivery drone. The city is represented as a 2D grid where some cells are obstacles (buildings), and others have different "air turbulence" levels that increase the battery cost of flying through them.
### The Scenario
Your drone starts at a Distribution Center ($S$) and must reach a Customer Location ($G$).
* Uninformed Search: The drone has no map of turbulence and needs the shortest path in terms of steps.
* Informed/Cost Search: The drone receives a "wind map" and must minimize total energy consumption.
* Long-term Planning: The drone needs to visit multiple customers in one trip, requiring a Genetic Algorithm.
## Tasks
1. BFS: Find the path with the minimum number of steps.

2. UCS: Find the path that minimizes total energy cost (considering wind turbulence).

3. IDS: Implement memory-efficient depth-based search.

4. Genetic Algorithm: Optimize the sequence for visiting 5 distinct delivery stops.

In [ ]:
# @title Assignment: Urban Drone Delivery Optimization (2026)
# @markdown This notebook contains the framework for the Metropolis Drone Routing problem.
# @markdown Fill in the sections marked "TODO".

import collections
import heapq
import random
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 1. City Environment & Visualization
# ==========================================

class CityMap:
    def __init__(self, size=10):
        self.rows = size
        self.cols = size
        self.grid = np.zeros((size, size))
        self.costs = np.ones((size, size))

        # ---setting ---
        # high-cost region
        for c in range(0, 10):
            self.costs[4, c] = 10.0  # 整行都是高能耗

        # barriers
        buildings = [(4,0), (4,1), (4,2), (4,7), (4,8), (4,9)]
        for r, c in buildings:
            self.grid[r, c] = 1

    def get_neighbors(self, pos):
        r, c = pos
        neighbors = []
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]: # Up, Down, Left, Right
            nr, nc = r + dr, c + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr, nc] == 0:
                neighbors.append((nr, nc))
        return neighbors

def visualize_path(city_map, path=None, title="Drone Navigation"):
    plt.figure(figsize=(8,8))
    # Plot grid and costs
    plt.imshow(city_map.costs, cmap='Blues', alpha=0.3)
    plt.imshow(city_map.grid, cmap='Greys', alpha=0.8)

    if path:
        path_pts = np.array(path)
        plt.plot(path_pts[:, 1], path_pts[:, 0], color='red', linewidth=2, marker='o', markersize=4)

    plt.title(title)
    plt.grid(True)
    plt.show()

# ==========================================
# 2. Search Algorithms (Student Implementation)
# ==========================================

def breadth_first_search(city_map, start, goal):
    """TASK 1: Find path with fewest steps using a FIFO queue."""
    queue = collections.deque([[start]])
    visited = {start}

    while queue:
        path = queue.popleft()
        current = path[-1]
        if current == goal:
			return path

        # TODO: Implement BFS neighbor expansion
        pass
    return None

def uniform_cost_search(city_map, start, goal):
    """TASK 2: Find path with minimum battery cost using a Priority Queue."""
    pq = [(0, [start])] # (cumulative_cost, path)
    visited = {} # pos: min_cost_to_reach

    while pq:
        cost, path = heapq.heappop(pq)
        current = path[-1]

        if current == goal:
			return path, cost

        # TODO: Implement UCS logic using city_map.costs[r, c]
        pass
    return None, float('inf')

def iterative_deepening_search(city_map, start, goal, max_depth=50):
    """TASK 3: DFS-based search with incremental depth limits."""
    def dls(current, goal, depth, path, visited):
        if current == goal: return path
        if depth <= 0: return None
        # TODO: Implement Depth Limited Search logic
        return None

    for depth in range(max_depth):
        result = dls(start, goal, depth, [start], {start})
        if result: return result
    return None

# ==========================================
# 3. Evolutionary Search (Genetic Algorithm)
# ==========================================

class GeneticRouter:
    """TASK 4: Optimize the order of 5 delivery stops (TSP variant)."""
    def __init__(self, stops):
        self.stops = stops
        self.pop_size = 50
        self.mutation_rate = 0.2

    def fitness(self, route_indices):
        """Calculate inverse of total Euclidean distance."""
        # TODO: Calculate sum of Euclidean distances between stops in route_indices
        return 1.0 / (total_dist + 1e-6)

    def evolve(self, generations=100):
        # Initial Population: Random permutations
        pop = [random.sample(range(len(self.stops)), len(self.stops)) for _ in range(self.pop_size)]

        for gen in range(generations):
            # TODO: Selection, Crossover (Ordered), and Mutation
            pass
# ==========================================
# 4. Execution
# ==========================================

# Constants for the Assignment
START_POS = (0, 0)
GOAL_POS = (9, 9)
DELIVERY_STOPS = [(1, 8), (3, 1), (6, 4), (7, 8), (9, 1)] # For GA Task

# Initialize Environment
metropolis = CityMap(size=10)

print("--- Testing BFS ---")
path_bfs = breadth_first_search(metropolis, START_POS, GOAL_POS)
if path_bfs:
    # calculate cost of BFS
    bfs_energy = sum(metropolis.costs[r, c] for r, c in path_bfs)
    print(f"BFS Total Steps: {len(path_bfs)}")
    print(f"BFS Actual Energy Cost: {bfs_energy}")
    visualize_path(metropolis, path_bfs, "BFS Path (Ignores Costs)")

print("--- Testing UCS ---")
path_ucs, total_cost = uniform_cost_search(metropolis, START_POS, GOAL_POS)
print(f"UCS Total Battery Cost: {total_cost}")
if path_ucs: visualize_path(metropolis, path_ucs, "UCS Result (Minimize Battery)")

print("--- Testing GA ---")
router = GeneticRouter(DELIVERY_STOPS)
best_route = router.evolve()
print(f"Best delivery sequence (indices): {best_route}")

In [ ]:
import collections
import heapq
import random
import numpy as np
import matplotlib.pyplot as plt
import math

# ==========================================
# 1. City Environment & Visualization
# ==========================================

class CityMap:
    def __init__(self, size=10):
        self.rows = size
        self.cols = size
        self.grid = np.zeros((size, size))
        self.costs = np.ones((size, size))

        # ---setting ---
        # high-cost region
        for c in range(0, 10):
            self.costs[4, c] = 10.0  # 整行都是高能耗

        # barriers
        buildings = [(4,0), (4,1), (4,2), (4,7), (4,8), (4,9)]
        for r, c in buildings:
            self.grid[r, c] = 1

    def get_neighbors(self, pos):
        r, c = pos
        neighbors = []
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]: # Up, Down, Left, Right
            nr, nc = r + dr, c + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr, nc] == 0:
                neighbors.append((nr, nc))
        return neighbors

def visualize_path(city_map, path=None, title="Drone Navigation"):
    plt.figure(figsize=(8,8))
    # Plot grid and costs
    plt.imshow(city_map.costs, cmap='Blues', alpha=0.3)
    plt.imshow(city_map.grid, cmap='Greys', alpha=0.8)

    if path:
        path_pts = np.array(path)
        plt.plot(path_pts[:, 1], path_pts[:, 0], color='red', linewidth=2, marker='o', markersize=4)

    plt.title(title)
    plt.grid(True)
    plt.show()

# ==========================================
# 2. Search Algorithms (Student Implementation)
# ==========================================

def breadth_first_search(city_map, start, goal):
    """TASK 1: Find path with fewest steps using a FIFO queue."""
    queue = collections.deque([[start]])
    visited = {start}

    while queue:
        path = queue.popleft()
        current = path[-1]

        if current == goal:
            return path

        # Expand neighbors
        for neighbor in city_map.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = list(path)
                new_path.append(neighbor)
                queue.append(new_path)

    return None

def uniform_cost_search(city_map, start, goal):
    """TASK 2: Find path with minimum battery cost using a Priority Queue."""
    pq = [(0, [start])] # (cumulative_cost, path)
    visited = {start: 0} # pos: min_cost_to_reach

    while pq:
        cost, path = heapq.heappop(pq)
        current = path[-1]

        if current == goal:
            return path, cost

        # Skip if we found a better path to this node already
        if cost > visited[current]:
            continue

        # Expand neighbors with cost consideration
        for neighbor in city_map.get_neighbors(current):
            # Calculate new cost to reach neighbor
            r, c = neighbor
            new_cost = cost + city_map.costs[r, c]

            # If this is the first time visiting neighbor or found a cheaper path
            if neighbor not in visited or new_cost < visited[neighbor]:
                visited[neighbor] = new_cost
                new_path = list(path)
                new_path.append(neighbor)
                heapq.heappush(pq, (new_cost, new_path))

    return None, float('inf')

def iterative_deepening_search(city_map, start, goal, max_depth=50):
    """TASK 3: DFS-based search with incremental depth limits."""
    def dls(current, goal, depth, path, visited):
        if current == goal:
            return path
        if depth <= 0:
            return None

        # Expand current node
        for neighbor in city_map.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                result = dls(neighbor, goal, depth-1, new_path, visited)
                if result:
                    return result
                visited.remove(neighbor)  # Backtrack

        return None

    # Iteratively increase depth limit
    for depth in range(max_depth):
        result = dls(start, goal, depth, [start], {start})
        if result:
            return result
    return None

# ==========================================
# 3. Evolutionary Search (Genetic Algorithm)
# ==========================================

class GeneticRouter:
    """TASK 4: Optimize the order of 5 delivery stops (TSP variant)."""
    def __init__(self, stops):
        self.stops = stops
        self.pop_size = 50
        self.mutation_rate = 0.2

    def calculate_distance(self, pos1, pos2):
        """Calculate Euclidean distance between two points."""
        r1, c1 = pos1
        r2, c2 = pos2
        return math.sqrt((r2 - r1)**2 + (c2 - c1)**2)

    def fitness(self, route_indices):
        """Calculate inverse of total Euclidean distance."""
        total_dist = 0.0

        # Add distance from start to first stop
        current_pos = self.stops[route_indices[0]]

        # Add distances between consecutive stops
        for i in range(1, len(route_indices)):
            next_pos = self.stops[route_indices[i]]
            total_dist += self.calculate_distance(current_pos, next_pos)
            current_pos = next_pos

        # Return fitness (higher is better)
        return 1.0 / (total_dist + 1e-6)

    def ordered_crossover(self, parent1, parent2):
        """Ordered crossover for TSP."""
        size = len(parent1)
        start, end = sorted(random.sample(range(size), 2))

        child = [-1] * size

        # Copy segment from parent1
        child[start:end+1] = parent1[start:end+1]

        # Fill remaining positions from parent2
        parent2_ptr = 0
        for i in range(size):
            if child[i] == -1:
                # Find next gene from parent2 that's not already in child
                while parent2[parent2_ptr] in child:
                    parent2_ptr += 1
                child[i] = parent2[parent2_ptr]

        return child

    def mutate(self, individual):
        """Swap mutation."""
        if random.random() < self.mutation_rate:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
        return individual

    def evolve(self, generations=100):
        # Initial Population: Random permutations
        pop = [random.sample(range(len(self.stops)), len(self.stops))
               for _ in range(self.pop_size)]

        best_fitness_history = []

        for gen in range(generations):
            # Calculate fitness for all individuals
            fitness_scores = [self.fitness(ind) for ind in pop]
            best_idx = np.argmax(fitness_scores)
            best_fitness = fitness_scores[best_idx]
            best_fitness_history.append(best_fitness)

            # Tournament selection
            new_pop = []
            for _ in range(self.pop_size):
                # Randomly select 3 individuals for tournament
                tournament = random.sample(list(enumerate(pop)), 3)
                # Select the one with highest fitness
                winner_idx = max(tournament, key=lambda x: fitness_scores[x[0]])[0]
                new_pop.append(pop[winner_idx])

            # Crossover and mutation
            pop = []
            for i in range(0, self.pop_size, 2):
                parent1 = new_pop[i]
                parent2 = new_pop[(i + 1) % self.pop_size]

                child1 = self.ordered_crossover(parent1, parent2)
                child2 = self.ordered_crossover(parent2, parent1)

                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                pop.extend([child1, child2])

            # Keep population size consistent
            pop = pop[:self.pop_size]

            # Print progress every 20 generations
            if gen % 20 == 0:
                print(f"Generation {gen}: Best fitness = {best_fitness:.6f}")

        # Find best route
        fitness_scores = [self.fitness(ind) for ind in pop]
        best_idx = np.argmax(fitness_scores)
        best_route = pop[best_idx]

        # Plot fitness progress
        plt.figure(figsize=(8, 4))
        plt.plot(best_fitness_history)
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness')
        plt.title('Genetic Algorithm Convergence')
        plt.grid(True)
        plt.show()

        return best_route

# ==========================================
# 4. Execution
# ==========================================

# Constants for the Assignment
START_POS = (0, 0)
GOAL_POS = (9, 9)
DELIVERY_STOPS = [(1, 8), (3, 1), (6, 4), (7, 8), (9, 1)] # For GA Task

# Initialize Environment
metropolis = CityMap(size=10)

print("=== Testing BFS ===")
path_bfs = breadth_first_search(metropolis, START_POS, GOAL_POS)
if path_bfs:
    # Calculate actual energy cost (even though BFS ignores it)
    bfs_energy = sum(metropolis.costs[r, c] for r, c in path_bfs)
    print(f"BFS Path length: {len(path_bfs)} steps")
    print(f"BFS Actual Energy Cost: {bfs_energy:.1f}")
    visualize_path(metropolis, path_bfs, "BFS Path (Minimizes Steps, Ignores Costs)")

print("\n=== Testing UCS ===")
path_ucs, total_cost = uniform_cost_search(metropolis, START_POS, GOAL_POS)
if path_ucs:
    print(f"UCS Path length: {len(path_ucs)} steps")
    print(f"UCS Total Battery Cost: {total_cost:.1f}")
    visualize_path(metropolis, path_ucs, "UCS Result (Minimizes Battery Cost)")

print("\n=== Testing IDS ===")
path_ids = iterative_deepening_search(metropolis, START_POS, GOAL_POS, max_depth=50)
if path_ids:
    ids_energy = sum(metropolis.costs[r, c] for r, c in path_ids)
    print(f"IDS Path length: {len(path_ids)} steps")
    print(f"IDS Actual Energy Cost: {ids_energy:.1f}")
    print(f"IDS path same as BFS: {path_ids == path_bfs}")
    visualize_path(metropolis, path_ids, "IDS Path (Memory-efficient BFS)")

print("\n=== Testing Genetic Algorithm ===")
print("Delivery Stops (indices and positions):")
for i, pos in enumerate(DELIVERY_STOPS):
    print(f"  Stop {i}: position {pos}")

router = GeneticRouter(DELIVERY_STOPS)
best_route_indices = router.evolve(generations=100)

print(f"\nBest delivery sequence (indices): {best_route_indices}")
print("Best delivery sequence (positions):")
for i, idx in enumerate(best_route_indices):
    print(f"  Step {i+1}: Stop {idx} at {DELIVERY_STOPS[idx]}")

# Calculate total distance for best route
total_dist = 0
current_pos = DELIVERY_STOPS[best_route_indices[0]]
for i in range(1, len(best_route_indices)):
    next_pos = DELIVERY_STOPS[best_route_indices[i]]
    dist = router.calculate_distance(current_pos, next_pos)
    total_dist += dist
    current_pos = next_pos
print(f"Total Euclidean distance: {total_dist:.2f}")

# ==========================================
# 5. Comparison and Analysis
# ==========================================

print("\n" + "="*50)
print("SUMMARY AND COMPARISON")
print("="*50)

print("\n1. BFS vs UCS Comparison:")
if path_bfs and path_ucs:
    print(f"   BFS steps: {len(path_bfs)}, Energy cost: {bfs_energy:.1f}")
    print(f"   UCS steps: {len(path_ucs)}, Energy cost: {total_cost:.1f}")
    print(f"   UCS avoids high-cost row (cost=10): {'Yes' if 4 not in [r for r,_ in path_ucs] else 'No'}")
    print(f"   Energy saving of UCS over BFS: {bfs_energy - total_cost:.1f} ({((bfs_energy - total_cost)/bfs_energy*100):.1f}%)")

print("\n2. BFS vs IDS Comparison:")
if path_bfs and path_ids:
    print(f"   Same path found: {path_bfs == path_ids}")
    print(f"   Same length: {len(path_bfs) == len(path_ids)}")

print("\n3. Genetic Algorithm Performance:")
print(f"   Best route found: {best_route_indices}")
print(f"   This sequence minimizes total travel distance between stops.")

Single Choice Questions:

Question 1: In the context of Uniform Cost Search (UCS), the priority queue ensures optimality by always expanding:

A. The node with the fewest number of steps from the start.

B. The node with the lowest estimated total cost to the goal (using a heuristic).

C. The node with the lowest cumulative path cost from the start.

D. The node at the shallowest depth in the search tree.

Question 2: When analyzing the space complexity of BFS, the primary memory bottleneck is storing the:

A. Visited set, which can grow as O(b^d).

B. Priority queue, which stores nodes sorted by cost.

C. Frontier (open) list, which at its worst holds all nodes at depth *d*.

D. The entire search tree up to depth *d-1*.


Question 3: In a Genetic Algorithm for path planning, the crossover rate primarily controls:

A. The introduction of entirely new genetic material into the population.

B. The speed at which high-fitness solutions take over the population.

C. How often two parent solutions are combined to form offspring.

D. The randomness of the initial population generation.


Short Description Questions:

BFS vs. UCS Optimality: Under what specific map conditions (cost distributions) will BFS fail to find the optimal path that minimizes battery consumption? How does the Priority Queue mechanism in UCS ensure optimality?

Space Complexity: In a search tree with branching factor $b$ and solution depth $d$, where is the primary memory bottleneck for BFS? If you encountered a "Memory Error," why would Iterative Deepening Search (IDS) be a better choice?

Genetic Algorithm Stochasticity: Genetic Algorithms are heuristic and non-deterministic. Discuss the trade-off of the Mutation Rate: what happens to the population convergence if the rate is set to 0.9 versus 0.01?

Path Cost vs. Nodes Expanded: Compare the number of nodes visited by BFS and UCS on this specific map. Which algorithm is more "efficient" in terms of computation time versus path quality?

Industry Choice: If you were designing the routing engine for a commercial drone delivery service (like Amazon Prime Air), would you use UCS or a Genetic Algorithm for point-to-point navigation? Please explain the reason.